Авторы данной работы Matthew Zeiler и Rob Fergus ранее работали над задачей представления изображения в виде результата свертки. Я делал обзоры на их предыдущие статьи: [Deconvolutional Networks]($Deconvolutional Networks$) (2010), [Adaptive Deconvolutional Networks for Mid and High Level Feature Learning]($Adaptive Deconvolutional Networks for Mid and High Level Feature Learning$) (2011).

После появления [AlexNet]($ImageNet Classification with Deep Convolutional Neural Networks$) авторы решили применить разработанную ими методологию для анализа глубоких сверточных сетей. В данной работе делается попытка интерпретации активаций отдельных фильтров в сверточных сетях путем обратного "проецирования" этих фильтров на исходное изображение. Эту работу я понял не до конца, поэтому данный обзор является скорее приглашением порассуждать на эту тему, и *является обзором лишь первой части статьи*.

### Рассматриваемая архитектура сети

Сверточная сеть, исследуемая авторами, похожа на AlexNet, но с двумя изменениями. Во-первых, размер фильтров первого слоя уменьшен с 11x11 с шагом 4x4 до 7x7 с шагом 2x2. Во-вторых, фильтры смежных слоев плотно связаны друг с другом, поскольку обучение модели не разделялось по 2 видеокартам, как в AlexNet.

Авторы утверждают, что инициализировали все веса значением $10^{-2}$.

> All weights are initialized to $10^{-2}$ and biases are set to 0.

С инициализацией, проведенной таким образом, сеть скорее всего не сможет обучиться. Возможно здесь имеется в виду, что веса инициализированы нормальным распределением со стандартным отклонением $10^{-2}$, как в AlexNet. Цитата из работы, посвященной AlexNet:

> We initialized the weights in each layer from a zero-mean Gaussian distribution with standard deviation 0.01.

Инициализация некоторых фильтров исправлялась вручную, чтобы отдельные фильтры не "доминировали" в картах признаков:

> Visualization of the first layer filters during training reveals that a few of them dominate, as shown in Fig. 6(a). To combat this, we renormalize each filter in the convolutional layers whose RMS value exceeds a fixed radius of $10^{-1}$ to this fixed radius. This is crucial, especially in the first layer of the model, where the input images are roughly in the [-128,128] range.

### Проецирование активаций фильтров на изображение

Сверточная сеть состоит из чередования слоев свертки, ReLU и max pooling. Подав на вход сети изображение, мы получим на выходе каждого слоя карту признаков, состоящую из значений, выданных несколькими фильтрами. Далее мы можем выбрать интересующий нас фильтр, максимальное значение активации в этом фильтре и занулить все остальные элементы массива, кроме выбранного. Мы получим карту признаков с одним ненулевым элементом. Дальнейшая задача состоит в том, чтобы спроецировать этот массив обратно на пространство входных изображений. Для этого мы проходим по сети в обратном направлении, заменяя каждый слой (свертка, ReLU, max pooling) на слой, действующий в обратном направлении. Авторы делают это следующим образом:

- Свертка заменяется на транспонированную свертку с теми же весами.
- ReLU заменяется на ReLU
- Max pooling при "прямом проходе" "запоминает" координаты выбранных точек с максимальными значениями, сохраняя их в массиве "switches", и после этого при "обратном проходе" помещает значения по тем же координатам

Схема из статьи:

<img src="assets/deconvnet.jpg" width="400" align="center">

Данная схема во всем (кроме ReLU) аналогична той, что использовалась в предыдущей работе тех же авторов. В [обзоре на нее]($Adaptive Deconvolutional Networks for Mid and High Level Feature Learning$) я более подробно описывал принцип работы данной схемы. Конечно, выполняемые преобразования не являются строго обратными преобразованиями, например невозможно инвертировать ReLU с помощью ReLU.

Транспонированная свертка также не является обратной сверткой. В [данной статье]($Транспонированная и обратная свертка$) я подробно разбирал разницу между ними в теории и на практике и пришел к выводу, что методы оптимизации (такие как Adam) справляются с задачей обращения операции свертки намного точнее, чем транспонирование. Но возможно авторы и не ставили своей целью в точности восстановить входные данные слоя по выходным? Используемый метод в статье обосновывается очень кратко и не до конца понятно:

> The convnet uses relu non-linearities, which rectify the feature maps thus ensuring the feature maps are always positive. *To obtain valid feature reconstructions at each layer (which also should be positive)*, we pass the reconstructed signal through a relu non-linearity.

> Projecting down from higher layers uses the switch settings generated by the max pooling in the convnet on the way up. As these switch settings are peculiar to a given input image, *the reconstruction obtained from a single activation thus resembles a small piece of the original input image, with structures weighted according to their contribution toward to the feature activation*. Since the model is trained discriminatively, they implicitly show which parts of the input image are discriminative. Note that these projections are not samples from the model, since there is no generative process involved.

### Визуализация результатов

Раздел, посвященный визуализации результатов проецирования, также оставляет вопросы, потому что комментарии к иллюстрациям в некоторых местах двусмысленны. По-видимому, каждый блок соответствует одному сверточному фильтру сети, для которого выбрано 9 наибольших активаций среди всего тестового датасета.

<img src="assets/deconvnet2.jpg" width="700" align="center">

Можно попытаться найти что-то общее в каждом слое. Например, фильтры второго слоя соответствуют простым формам (круги, линии), фильтры пятого слоя соответствуют более сложным, но интерпретируемым понятиям (собаки, цветы и пр.).

Кроме того, авторы показывают, что в процессе обучения сети проекции активаций фильтров становятся боле "четкими" и выделяют некоторые конкретные объекты, причем чем больше номер слоя в сети, тем медленнее идет этот процесс. На иллюстрации можно видеть, что фильтры нижних слоев "сходятся" быстрее, чем верхних (по горизонтали - номер эпохи из последовательности [1,2,5,10,20,30,40,64]):

<img src="assets/deconvnet3.jpg" width="800" align="center">

Также авторы исследуют устойчивость фильтров к смещению, масштабированию и поворотам изображения. Для этого были выбраны 5 изображений. Первая строка соответствует смещениям, вторая масштабированию, третья поворотам. В первом столбце показаны сами изображения. Во втором столбце показана зависимость евклидова расстояния между картами признаков первого слоя (ось Y) от степени трансформации (ось X) для всех 5 изображений (цвет графика). В третьем столбце показано то же самое для 7-го слоя.  В четвертом столбце показана вероятность, присвоенная моделью верному классу (то есть степень правильности ответа модели) в зависимости от степени трансформации.

<img src="assets/deconvnet4.jpg" width="1000" align="center">

Можно сделать вывод о том, что седьмой слой существенно более инвариантен к смещению, масштабированию и повороту, чем первый слой. К повороту как седьмой, так и первый слой проявляют большую чувствительность. Как классификатор нейронная сеть более устойчива к смещению и масштабированию, чем к повороту.

Конечно, это является естественным следствием процесса обучения сети. Обучение проводилось с аугментациями (хотя явно в статье это не говорится, наверняка это так). Поэтому сеть учится выдавать ответ, инвариантный к аугментациям, то есть к сдвигу и масштабированию. Непонятно был ли включен поворот в список аугментаций. Поскольку последние слои ближе к выходу, они более инвариантны к этим преобразованиям, чем первые слои.